In [7]:
%pip install langchain langchain-huggingface chroma transformers langchain-core langchain-community pypdf

Defaulting to user installation because normal site-packages is not writeable
     ------------------------------------ 298.0/298.0 kB 836.4 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [14]:
%pip install sentence-transformers

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [16]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain_huggingface import HuggingFaceEndpoint

In [26]:
import json
creds = {}
with open("creds.json") as f:
    creds = json.load(f)

In [21]:
loader = PyPDFLoader(creds["filepath"])
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=30)
docs = text_splitter.split_documents(documents)
print(docs[0].page_content)

Unit -III Map Reduce And Spark FrameworkMap Reduce: Anatomy of a Map Reduce job Run, Failure, Job Scheduling, Shuffle and sort, Task Exception, Map Reduce Types and formats, Map Reduce feactures, Map Reduce programming: I/O formats, Map side join, Reduce side join,Secondary sorting, Pipelining Map reduce jobs.Spark frameworks: Introduction to GPU Computing,CUDA programming model, CUDA API, Simple matrix,Multiplication in CUDA, CUDA memory model, Shared memory matrix multiplication,Additional CUDA API Features.Anatomy of a Map Reduce job RunThe term "big data analytics" refers to the application of cutting-edge analytical methods to very large, diversified big data sets, which may comprise structured, semi-structured, and unstructured data from many sources and range in size from terabytes to zettabytes. The high volume, high velocity, and high variety of big data are its characteristics. Due to the influence of artificial intelligence (AI), mobile technology, social media, and the Inte

In [23]:
embeddings = HuggingFaceBgeEmbeddings(model_name="all-MiniLM-L6-v2")

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [24]:
# create vector store if not exists
db = Chroma.from_documents(docs, embeddings, persist_directory="db")
db.persist()

C:\Users\navee\AppData\Local\Temp\ipykernel_10352\738916483.py:3: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()


In [25]:
retriever = db.as_retriever(search_kwargs={"k": 2},
                            search_type="similarity",
                            )
retriever.get_relevant_documents("what is map?")

C:\Users\navee\AppData\Local\Temp\ipykernel_10352\3761223157.py:4: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents("what is map?")


[Document(metadata={'page': 0, 'source': 'D:/NK_Studies/Sem5/BDA/bda3.pdf'}, page_content='Unit -III Map Reduce And Spark FrameworkMap Reduce: Anatomy of a Map Reduce job Run, Failure, Job Scheduling, Shuffle and sort, Task Exception, Map Reduce Types and formats, Map Reduce feactures, Map Reduce programming: I/O formats, Map side join, Reduce side join,Secondary sorting, Pipelining Map reduce jobs.Spark frameworks: Introduction to GPU Computing,CUDA programming model, CUDA API, Simple matrix,Multiplication in CUDA, CUDA memory model, Shared memory matrix multiplication,Additional CUDA API Features.Anatomy of a Map Reduce job RunThe term "big data analytics" refers to the application of cutting-edge analytical methods to very large, diversified big data sets, which may comprise structured, semi-structured, and unstructured data from many sources and range in size from terabytes to zettabytes. The high volume, high velocity, and high variety of big data are its characteristics. Due to t

In [31]:
llm = HuggingFaceEndpoint(
        repo_id="mistralai/Mistral-7B-Instruct-v0.2",
        max_length=500,
        temperature=0.8,
        huggingfacehub_api_token=creds["hf_token"],
    )
# from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
# callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
# llm = LlamaCpp(
#     model_path=creds["modelpath"],
#     callback_manager=callback_manager,
#     verbose=True,  # Verbose is required to pass to the callback manager
#     max_context_length=4096,
#     max_new_tokens=1024,
#     temperature=0.75,
#     top_p=1,
#     n_gpu_layers=1,
#     n_batch=512,
#     n_ctx=4096
# )

WARNING! max_length is not default parameter.
                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.


In [28]:
from langchain import PromptTemplate

template = """
You are a big data analytics expert. These Human will ask you a questions and doubts on the topic. 
Use following piece of context to answer the question. 
If you don't know the answer, just say you don't know. 
Keep the answer within 5 sentences and concise.

Context: {context}
Question: {question}
Answer: 

"""

prompt = PromptTemplate(
  template=template, 
  input_variables=["context", "question"]
)

In [32]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

rag_chain = (
  {"context": retriever,  "question": RunnablePassthrough()} 
  | prompt 
  | llm
  | StrOutputParser() 
)

In [33]:
qn = input("What is it you want? >> ")
res = rag_chain.invoke(qn)
print(res)

Map is a process in Apache Hadoop's MapReduce framework that operates on small data subsets, called input splits. It applies a user-defined function, called a mapper, to each input record to generate output records. The output records are then shuffled and sorted based on their keys before being processed by the reduce phase.
